## 读取数据

In [1]:
import torch
from torch import nn
import gzip
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    """加载MNIST数据集"""
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte.gz')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte.gz')

    with gzip.open(labels_path, 'rb') as lbpath:
        struct.unpack('>II', lbpath.read(8))
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        struct.unpack('>IIII', imgpath.read(16))
        images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels),28,28)

    return images, labels



# 数据集划分
def data_split(images, labels, ratio):
    
    total_len = images.shape[0]
    offset = int(total_len * ratio)

    val_img = images[:offset][:]
    val_lb = labels[:offset]

    train_img = images[offset:][:]
    train_lb = labels[offset:]

    return train_img, train_lb, val_img, val_lb    

# 读取训练集和测试集数据
[images, labels] = load_mnist('./MNIST', kind='train')
[test_img, test_lb] = load_mnist('./MNIST',kind='test')
train_img, train_lb, val_img, val_lb = data_split(images, labels, 1/6)

#random_numbers = np.random.randint(50000, size=(2000, ))
train_img=train_img#[random_numbers]
# 对标签进行热编码
one_hot_train_lb = np.eye(10)[train_lb]
one_hot_val_lb = np.eye(10)[val_lb]
one_hot_test_lb= np.eye(10)[test_lb]

# 打印查看数据集格式
print('训练集图像格式为:', train_img.shape, '训练集标签格式为:', train_lb.shape,'热编码训练集标签格式为:', one_hot_train_lb.shape)
print('验证集图像格式为:', val_img.shape, '验证集标签格式为:', val_lb.shape,'热编码验证集标签格式为:', one_hot_val_lb.shape)
print('测试集图像格式为:', test_img.shape, '测试集标签格式为:', test_lb.shape,'热编码测试集标签格式为:', one_hot_test_lb.shape)


训练集图像格式为: (50000, 28, 28) 训练集标签格式为: (50000,) 热编码训练集标签格式为: (50000, 10)
验证集图像格式为: (10000, 28, 28) 验证集标签格式为: (10000,) 热编码验证集标签格式为: (10000, 10)
测试集图像格式为: (10000, 28, 28) 测试集标签格式为: (10000,) 热编码测试集标签格式为: (10000, 10)


In [2]:
#calc hog as feature
import cv2
hog = cv2.HOGDescriptor(_winSize=(28,28),_blockSize=(14,14), _blockStride=(7,7), _cellSize=(7,7),_nbins=(9))
def get_hog(x):
    res = hog.compute(x)
    return res
new_train=[]
for i in range(train_img.shape[0]):
    new_train.append(get_hog(train_img[i]))
train_img=np.array(new_train)
new_test=[]
for i in range(test_img.shape[0]):
    new_test.append(get_hog(test_img[i]))
test_img=np.array(new_test)

print(train_img.shape)
print(test_img.shape)

(50000, 324)
(10000, 324)


## 定义网络

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):

        super().__init__()
        self.w1=nn.Linear(324, 10)
        self.drop=nn.Dropout(p=0.5)
        self.relu=nn.ReLU()


    def forward(self, x):
        x = self.w1 (x)
        x = self.relu(x)
        return x

model = NeuralNetwork()

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



## 定义训练

In [4]:

batch_size = 100
epochs = 20
batch_num=int(train_img.shape[0]/batch_size)
size = len(train_img)

model.train()
for t in range(epochs):
    
    correct=0.
    train_mean_loss=0.

    for batch in range(batch_num):
        X=train_img[batch*batch_size:(batch+1)*batch_size,]
        y=one_hot_train_lb[batch*batch_size:(batch+1)*batch_size,:]

        X=torch.tensor(X, dtype=torch.float32)
        y=torch.tensor(y, dtype=torch.float32)

        # Compute prediction and loss
        pred = model(X).type(torch.float)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).mean().item()
        train_mean_loss+= loss.item()

    train_mean_loss /= batch_num
    correct /= batch_num
    
    print(f" Epoch:{t+1}, loss: {train_mean_loss:>8f},  Accuracy: {(100*correct):>0.1f}%")


 Epoch:1, loss: 1.329697,  Accuracy: 72.0%
 Epoch:2, loss: 0.812622,  Accuracy: 76.9%
 Epoch:3, loss: 0.704620,  Accuracy: 79.0%
 Epoch:4, loss: 0.655989,  Accuracy: 80.7%
 Epoch:5, loss: 0.627537,  Accuracy: 81.9%
 Epoch:6, loss: 0.608587,  Accuracy: 82.8%
 Epoch:7, loss: 0.594914,  Accuracy: 83.5%
 Epoch:8, loss: 0.584531,  Accuracy: 84.0%
 Epoch:9, loss: 0.576382,  Accuracy: 84.4%
 Epoch:10, loss: 0.569848,  Accuracy: 84.7%
 Epoch:11, loss: 0.564514,  Accuracy: 85.0%
 Epoch:12, loss: 0.560048,  Accuracy: 85.2%
 Epoch:13, loss: 0.556253,  Accuracy: 85.4%
 Epoch:14, loss: 0.552988,  Accuracy: 85.6%
 Epoch:15, loss: 0.550143,  Accuracy: 85.7%
 Epoch:16, loss: 0.547652,  Accuracy: 85.9%
 Epoch:17, loss: 0.545431,  Accuracy: 86.0%
 Epoch:18, loss: 0.543432,  Accuracy: 86.1%
 Epoch:19, loss: 0.541635,  Accuracy: 86.2%
 Epoch:20, loss: 0.540002,  Accuracy: 86.3%


## 定义测试

In [5]:
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
        X=torch.tensor(test_img, dtype=torch.float32)
        y=torch.tensor(one_hot_test_lb, dtype=torch.float32)
        pred = model(X)
        test_loss = np.mean(loss_fn(pred, y).item())
        correct = (pred.argmax(1) == y.argmax(1)).type(torch.float).mean().item()

print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")


Test Accuracy: 86.8%, Test Avg loss: 0.530638 

